In [1]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 911kB 5.4MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/d

In [2]:
from bs4 import BeautifulSoup
import re, time
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [3]:
#keyword scraping approach, combined use of selenium and beautiful soup

#tweet_url function will return correct url to target

#notes on expected format of inputs, strings
#date format: YYYY-MM-DD
#loc: City
#radius: 150mi or 100mi
#function will read keyword as hashtag automatically (in respect of Twitter robots.txt file)

def tweet_url(keyword, loc, beg_date, end_date, radius):
    targeting = 'https://twitter.com/search?q=' + keyword + '%20near%3A' + loc + '%20since%3A' + beg_date + '%20until%3A' + end_date + '%20within%3A' + radius + '&src=recent_search_click&f=live'
    return targeting

In [4]:
#trying to get all html content of twitter search page

#manual crawl function tries to collect all page info as you scroll down
#manual because we set number of scrolls
#limited by number of scrolls set, might not be end of page yet if set too low, say, below 100

def manual_crawl_page(url, n):
    #loading search page
    driver = webdriver.Chrome('chromedriver', options=chrome_options)
    driver.get(url)
    driver.implicitly_wait(15)
    
    #empty list where we will append all html content collected as we scroll down
    html=[]
    
    #scroll for n seconds
    for i in range(n):
        #search for all relevant contents of twitter search page
        list_items = driver.find_elements_by_class_name('css-1dbjc4n')
        #collecting html content
        html.append(list_items[0].get_attribute('innerHTML'))
        
        #to scroll down
        elem = driver.find_element_by_tag_name('body')
        elem.send_keys(Keys.END)
        
        time.sleep(np.random.randint(1,3))
    
    #parsing each html content collected
    soup = []
    for i in html:    
        soup += BeautifulSoup(i,'lxml')
        
    return soup

In [5]:
#auto crawl function tries to collect all page info as it automatically scrolls down until end of page
#note: takes some time to finish, also extracts fewer tweets. might be due to pace of scrolling

def auto_crawl_page(url):
    #loading search page
    driver = webdriver.Chrome('chromedriver', options=chrome_options)
    driver.get(url)    
    
    time.sleep(np.random.randint(1,3))  
    
    #empty list where we will append all html content collected as we scroll down
    html = []
    
    #calculates length of page
    lenpage = driver.execute_script("var lenpage=document.body.scrollHeight;return lenpage;")      
    
    #to iterate until bottom of page
    match = False
    while(match==False):
        #implicitly wait to load page
        driver.implicitly_wait(10)
        
        #search for all relevant contents of twitter search page
        list_items = driver.find_elements_by_class_name('css-1dbjc4n')
        #collecting html content
        html.append(list_items[0].get_attribute('innerHTML'))
        
        #re-assigns value of length of page
        lastcount = lenpage
        
        time.sleep(np.random.randint(10,30))
        
        #to scroll down
        elem = driver.find_element_by_tag_name('body')
        elem.send_keys(Keys.END)
        
        #re-calculates length of page
        lenpage = driver.execute_script("var lenpage=document.body.scrollHeight;return lenpage;")
        
        #criteria that evaluates if we are at the bottom of page already
        if lastcount==lenpage:
            match=True
    
    #parsing each html content collected
    soup = []
    for i in html:    
        soup += BeautifulSoup(i,'lxml')
        
    return soup

In [6]:
#function extracting tweet details

def tweetsrc_data(soup):
    #get all tweets from each parsed html
    tweets = []
    for a in soup:
        temp = a.find_all('article')
        for b in temp:
            tweets.append(b)
    
    #empty lists where we will append tweet info
    post = []
    timestamp = []
    retweet = []
    like = []
    reply = []

    #to iterate over tweets collected, if-else is to account for unexpected errors
    for i in tweets:
        #post
        post.append(re.sub('(.*)\@(\w+)\W\d+h?\s?(Mar|Apr|May)?','',i.get_text())) #post
        
        #timestamp
        if i.find('time')==None:
            timestamp.append(np.nan)
        elif len(i.find('time')['datetime'])==19:
            timestamp.append(i.find('time')['datetime'])
        elif len(i.find('time')['datetime'])==24:
            timestamp.append(i.find('time')['datetime'].replace('.000Z',''))
        
        #retweets
        if i.find('div',{'data-testid':'retweet'})==None:
            retweet.append(0)
        else:
            retweet.append(i.find('div',{'data-testid':'retweet'}).get_text()) #retweet
        
        #likes
        if i.find('div',{'data-testid':'like'})==None:
            like.append(0)
        else:
            like.append(i.find('div',{'data-testid':'like'}).get_text())#like
        
        #replies
        if i.find('div',{'data-testid':'reply'})==None:
            reply.append(0)
        else:
            reply.append(i.find('div',{'data-testid':'reply'}).get_text()) #reply
    
    #making dataframe of all tweet info
    df = pd.DataFrame({'timestamp':timestamp, 'post':post, 'reply':reply, 'like':like, 'retweet':retweet})
    
    #some tweets don't have necessary tag for tweet stats so it returns blanks, replacing them with 0
    df['reply'] = df['reply'].replace('',0)
    df['like'] = df['like'].replace('',0)
    df['retweet'] = df['retweet'].replace('',0)
    
    #drop duplicate tweets which may be collected repeatedly due to overlaps in page info while scrolling
    df = df.drop_duplicates(subset=['timestamp', 'post', 'reply', 'like', 'retweet'], keep='first')
    
    #making sure indices are in order
    df = df.reset_index()
    df = df.drop(columns='index')
    
    return df

In [8]:
#sample data
sample_url = tweet_url('shopping', 'jakarta', '2021-07-17', '2021-07-18', '150mi')
sample_soup = manual_crawl_page(sample_url, 50)

tweetsrc_data(sample_soup)

,timestamp,post,reply,like,retweet
0,2021-07-17T21:08:17,WANITA\nMenangis bukan karena lemah\nTapi kare...,0,0,0
1,2021-07-17T20:49:29,Replying to @radenraufLawakan suami. Karna kal...,0,0,0
2,2021-07-17T19:11:11,kira kira malem2 gini menjelang pagi masih ada...,0,0,0
3,2021-07-17T19:09:59,Dicari Reseller Masker Wajah Alami di Wonosari...,0,0,0
4,2021-07-17T18:59:21,Sandal Casual Pria Hexpreme #Sandal #sandalPri...,0,0,0
5,2021-07-17T18:44:46,shopping w my mom is fun she be like “yo te lo...,0,1,0
6,2021-07-17T18:26:03,Answer These Shopping Math Questions To Find O...,0,0,0
7,2021-07-17T18:20:58,Replying to @tawan_vihokrdulu window shopping ...,0,0,1
8,2021-07-17T18:16:03,AnisaChiBi teteh anchib...boleh minta follback...,0,0,0
9,2021-07-17T17:38:43,"#ETC See if i can HODL my futures ETCUSDT, hop...",0,0,0


In [10]:
#full data in 2021
url = tweet_url('shopping', 'Jakarta', '2021-01-01', '2021-07-18', '150mi')
soup = manual_crawl_page(url, 1000)
data = tweetsrc_data(soup)

In [11]:
data.head()

,timestamp,post,reply,like,retweet
0,2021-07-17T21:08:17,WANITA\nMenangis bukan karena lemah\nTapi kare...,0,0,0
1,2021-07-17T18:26:03,Answer These Shopping Math Questions To Find O...,0,0,0
2,2021-07-17T18:20:58,Replying to @tawan_vihokrdulu window shopping ...,0,0,1
3,2021-07-17T17:38:43,"#ETC See if i can HODL my futures ETCUSDT, hop...",0,0,0
4,2021-07-17T17:09:24,Hallo tweps Dk_Onlineshop menjual aneka ragam ...,0,0,0


In [12]:
data.shape

(403, 5)

In [13]:
#remove irrelevant records
data = data[~data['timestamp'].isnull()]

#prettify indices
data = data.reset_index()
data = data.drop(columns='index')

#fixing timestamps and making new date and time columns
data['timestamp'] = data['timestamp'].apply(str)
data['date'] = pd.to_datetime(data['timestamp'].apply(lambda x: x.split('T')[0])).dt.date
data['time'] = pd.to_datetime(data['timestamp'].apply(lambda x: x[11:])).dt.time
data['timestamp'] = data['date'].apply(str) + " " + data['time'].apply(str)
data['timestamp'] = pd.to_datetime(data['timestamp'], format='%Y-%m-%d %H:%M:%S')

#cleaning post content
data['post'] = data['post'].str.replace('\n',' ')

#to fix likes and retweets, removes K-- which indicates thousand values
def kLikes(row):
    if row['like'] == None:
        row['like'] = '0'
    if 'K' in row['like']:
        digits = re.findall('\d+', row['like'])
        if len(digits) == 1:
            return digits[0] + '000'
        elif len(digits) == 2:
            return digits[0] + digits[1] +'00'
    else:
        return row['like']

def kRetweets(row):
    if row['retweet'] == None:
        row['retweet'] = '0'
    if 'K' in row['retweet']:
        digits = re.findall('\d+', row['retweet'])
        if len(digits) == 1:
            return digits[0] + '000'
        elif len(digits) == 2:
            return digits[0] + digits[1] +'00'
    else:
        return row['retweet']

data['like'] = data['like'].astype('str')
data['retweet'] = data['retweet'].astype('str')
data['like'] =data.apply(kLikes, axis = 1)
data['retweet'] = data.apply(kRetweets, axis = 1)
data['like'] = data['like'].astype('int64')
data['retweet'] = data['retweet'].astype('int64')

In [14]:
data.head()

,timestamp,post,reply,like,retweet,date,time
0,2021-07-17 21:08:17,WANITA Menangis bukan karena lemah Tapi karena...,0,0,0,2021-07-17,21:08:17
1,2021-07-17 18:26:03,Answer These Shopping Math Questions To Find O...,0,0,0,2021-07-17,18:26:03
2,2021-07-17 18:20:58,Replying to @tawan_vihokrdulu window shopping ...,0,0,1,2021-07-17,18:20:58
3,2021-07-17 17:38:43,"#ETC See if i can HODL my futures ETCUSDT, hop...",0,0,0,2021-07-17,17:38:43
4,2021-07-17 17:09:24,Hallo tweps Dk_Onlineshop menjual aneka ragam ...,0,0,0,2021-07-17,17:09:24


In [15]:
data.shape

(402, 7)

In [17]:
#write data to CSV file
data.to_csv("shopping_tweet.csv", index = False, header = True)